# On the Identification of Pulsar Stars

A DSCI 100 Project

Lucas Kuhn, Sophia Zhang, Michael Cheung

## Introduction
"Pulsar" neutron stars produce radio emissions that are of great interest to astronomers and researchers. In this project, we set out to develop a model for classifying them distinctly from those that are "non-pulsar".

Our question to answer: "Are there characteristics of a pulsar candidate's curve profiles that serve as good predictors for pulsar and non-pulsar classification?".

We use the HTRU2 dataset from the UCI Machine Learning Repository, which contains summary statistics for observed pulsar candidates as they exhibit on both the Integrated Pulse (folded) Profile, as well as on the Dispersion Measure Signal-to-Noise Ratio (DM-SNR) curve.

## Preliminary Exploratory Data Analysis

In [1]:
# This serves as the set of packages that may be used for our final project

# Retrieve relevant packages for dataframes and visualization tools
import pandas as pd
import altair as alt
import sklearn

# Retrieve relevant packages for classification and modelling
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

# This dataset has more than 5000 obervations, we will need to allow for greater data plotting
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Data is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/00372/). Unzipping allows for reading the .CSV file into Jupyter Notebook.

In [2]:
# Using the extracted contents, read the .CSV file, identifying columns using the associated .ARFF alongside it
pulsar_data = pd.read_csv(
    "../data/HTRU2/HTRU_2.csv",
    # .CSV file contains no column headers, we provide these here
    names=[
        "Profile_mean",
        "Profile_stdev",
        "Profile_skewness",
        "Profile_kurtosis",
        "DM_mean",
        "DM_stdev",
        "DM_skewness",
        "DM_kurtosis",
        "class"
    ]
)

# As a POC, show the first 5 rows of the dataset
pulsar_data.head()

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [3]:
pulsar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Profile_mean      17898 non-null  float64
 1   Profile_stdev     17898 non-null  float64
 2   Profile_skewness  17898 non-null  float64
 3   Profile_kurtosis  17898 non-null  float64
 4   DM_mean           17898 non-null  float64
 5   DM_stdev          17898 non-null  float64
 6   DM_skewness       17898 non-null  float64
 7   DM_kurtosis       17898 non-null  float64
 8   class             17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [4]:
pulsar_data.dropna()
pulsar_data

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


From the "info" function, all observations have non-null variable values and the data is seen to be tidy as-is.

For ease of identification, we change 0s and 1s to readable word equivalents.

In [5]:
pulsar_data["class"] = pulsar_data["class"].replace({0: "non-pulsar", 1: "pulsar"}).convert_dtypes()
pulsar_data

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,non-pulsar
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,non-pulsar
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,non-pulsar
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,non-pulsar
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,non-pulsar
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,non-pulsar
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,non-pulsar
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,non-pulsar
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,non-pulsar


In [6]:
# Keep one other column outside the target classification column
pulsar_data_count = pulsar_data[["class", "Profile_mean"]].groupby("class").count().reset_index()

# Rename the column to more accurately reflect its purpose
pulsar_data_count = pulsar_data_count.rename(columns={"Profile_mean": "count"})
pulsar_data_count = pulsar_data_count.assign(pct=pulsar_data_count["count"] / len(pulsar_data))
pulsar_data_count

,class,count,pct
0,non-pulsar,16259,0.908426
1,pulsar,1639,0.091574


In this dataset, around 91% of the observations are "non-pulsar", and about 9% are "pulsar".

We split into training and testing subsets, arbitrarily using 75% of the data training alongside a stratification of the data by target labels to preserve the distribution of "pulsar" and "non-pulsar" observations.

In [7]:
pulsar_data_train, pulsar_data_test = train_test_split(
    pulsar_data, train_size=0.75, stratify=pulsar_data["class"]
)

In [8]:
pulsar_data_train_count = pulsar_data_train[["class", "Profile_mean"]].groupby("class").count().reset_index()
pulsar_data_train_count = pulsar_data_train_count.rename(columns={"Profile_mean": "count"})
pulsar_data_train_count = pulsar_data_train_count.assign(pct=pulsar_data_train_count["count"] / len(pulsar_data_train))
pulsar_data_train_count

,class,count,pct
0,non-pulsar,12194,0.908441
1,pulsar,1229,0.091559


In [9]:
# We only evaluate the distribution of the testing set; we seek no further detail into the nature of the testing set past this
pulsar_data_test_count = pulsar_data_test[["class", "Profile_mean"]].groupby("class").count().reset_index()
pulsar_data_test_count = pulsar_data_test_count.rename(columns={"Profile_mean": "count"})
pulsar_data_test_count = pulsar_data_test_count.assign(pct=pulsar_data_test_count["count"] / len(pulsar_data_test))
pulsar_data_test_count

,class,count,pct
0,non-pulsar,4065,0.90838
1,pulsar,410,0.09162


To help summarize our training data, we can calculate the mean value of the "Profile_mean" and "DM_mean" columns.

In [10]:
# Obtain the mean of means for the training set
pulsar_data_train_means = pulsar_data_train[["class", "Profile_mean", "DM_mean"]].groupby("class").mean(numeric_only=True).reset_index()
pulsar_data_train_means

,class,Profile_mean,DM_mean
0,non-pulsar,116.603105,8.806431
1,pulsar,55.707441,50.283852


At a higher level, we can create a scatter plot of the data in the training set.

In [11]:
pulsar_data_train_means_plot = (
    alt.Chart(pulsar_data_train, title="Mean Comparison")
    .mark_point(opacity=0.4)
    .encode(
        x=alt.X("Profile_mean", title="Mean using the Integrated Pulse Profile", scale=alt.Scale(zero=False)),
        y=alt.Y("DM_mean", title="Mean using the DM-SNR Curve"),
        color=alt.Color("class", title="Classification"),
    )
    .configure_axis(labelFontSize=16, titleFontSize=16)
    .properties(width=400, height=400)
)
pulsar_data_train_means_plot

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

A clear distinguishable separation vertically between the classifications implies that these variables could be effective predictors for our model.

## Methods

We plan to train multiple models using various subsets of the summary-statistic columns as predictor variables, given how the pulses for a candidate that is truly pulsar are "remarkably constant over long periods of time" (Karastergiou et. al, 2011) and as such tend to reveal prominent insights on curve profiles. 

We can use scatter plots similar to above to qualitatively assess pairs of variables for any distinct separation or clustering of classified points. Plots like this provide an interface for human inspection, as used to train other models such as artificial neural networks (Eatough et al., 2010).

We will fit a K-Nearest Neighbours classifier model to our training data, using a GridSearchCV with 5-fold Cross-Validation to choose $k$. This allows us to efficiently test the validation accuracy at many values of $k$ and combat both overfitting and underfitting.

With our best model, we can assess its accuracy on the testing set using a confusion matrix, allowing us to more critically assess the accuracy based on whether false positives are more costly than false negatives.

## Expected Outcomes and Significance

Our expectation is to determine the variables necessary for an effective classification model of "pulsar" and "non-pulsar" candidates.

This may be useful in finding and identifying future "pulsar" neutron stars in more complicated models. It may also help narrow down future observations and disregard unlikely candidatesat earlier stages of investigation.

A future extension or question may include:
- Could a machine learning process be implemented extending on these variables for even greater accuracy?

# References

1. Eatough, R. P., Molkenthin, N., Kramer, M., Noutsos, A., Keith, M. J., Stappers, B. W., & Lyne, A. G. (2010). Selection of radio pulsar candidates using artificial neural networks. Monthly Notices of the Royal Astronomical Society, 407(4), 2443-2450.

2. Karastergiou, A., Roberts, S. J., Johnston, S., Lee, H. J., Weltevrede, P., & Kramer, M. (2011). A transient component in the pulse profile of PSR J0738− 4042. Monthly Notices of the Royal Astronomical Society, 415(1), 251-256.